Refs:
[tdidf post](https://marcellodesales.wordpress.com/2009/12/31/tf-idf-in-hadoop-part-1-word-frequency-in-doc/)
[google](https://code.google.com/p/hadoop-clusternet/wiki/RunningMapReduceExampleTFIDF)
[mr-job implementation](https://github.com/louismullie/tf-idf-emr)

###Job 1: Word Frequency in Doc

As mentioned before, the word frequency phase is designed in a Job whose task is to count the number of words in each of the documents in the input directory. In this case, the specification of the Map and Reduce are as follows:

    Map:
        Input: (document, each line contents)
        Output: (word@document, 1))
    Reducer
        n = sum of the values of for each key “word@document”
        Output: ((word@document), n)

###Job 2: Word Counts for Docs

The goal of this job is to count the total number of words for each document, in a way to compare each word with the total number of words. I’ve tried to implement a default InputFormat and I couldn’t find examples related to it. As I understood, the values could be read in the same format they are saved (Text, IntWritable), but I will keep it simple and use the same default InputFormat as before. Following the same definition as in part one, the specification of the Map and Reduce are as follows:

    Map:
        Input: ((word@document), n)
        Re-arrange the mapper to have the key based on each document
        Output: (document, word=n)
    Reducer
        N = totalWordsInDoc = sum [word=n]) for each document
        Output: ((word@document), (n/N))

###Job 3: Documents in Corpus and TF-IDF computation
   
The output shows each term at each document, and the number of its occurrence on the given document, accompanied by the total number of terms in the document. So, the final Mapper and Reducer were defined as follows:

    Map:
        Input: ((term@document), n/N)
        Output: (term, document=n/N)
    Reducer:
        D = total number of documents in corpus. This can be passed by the driver as a constant;
        d = number of documents in corpus where the term appears. 
        TFIDF = n/N * log(D/d);
        Output: ((word@document), d/D, (n/N), TFIDF)
    